In [1]:
import pandas as pd
import joblib
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [2]:
df=pd.read_csv("data/CTG.csv")
df

,FileName,Date,SegFile,b,e,LBE,LB,AC,FM,UC,...,C,D,E,AD,DE,LD,FS,SUSP,CLASS,NSP
0,Variab10.txt,12/1/1996,CTG0001.txt,240.0,357.0,120.0,120.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,9.0,2.0
1,Fmcs_1.txt,5/3/1996,CTG0002.txt,5.0,632.0,132.0,132.0,4.0,0.0,4.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,6.0,1.0
2,Fmcs_1.txt,5/3/1996,CTG0003.txt,177.0,779.0,133.0,133.0,2.0,0.0,5.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,6.0,1.0
3,Fmcs_1.txt,5/3/1996,CTG0004.txt,411.0,1192.0,134.0,134.0,2.0,0.0,6.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,6.0,1.0
4,Fmcs_1.txt,5/3/1996,CTG0005.txt,533.0,1147.0,132.0,132.0,4.0,0.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2124,S8001045.dsp,6/6/1998,CTG2127.txt,1576.0,3049.0,140.0,140.0,1.0,0.0,9.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,5.0,2.0
2125,S8001045.dsp,6/6/1998,CTG2128.txt,2796.0,3415.0,142.0,142.0,1.0,1.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
2126,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2127,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# Load your dataset into a DataFrame (assuming it's in a CSV file)
df = pd.read_csv('data/CTG.csv')

# Drop rows with null values
df.dropna(inplace=True)

# Define input features and target variable
X = df.drop(columns=['FileName', 'Date', 'SegFile', 'NSP'])
y = df['NSP']

# Convert labels to binary (0 and 1)
y = (y == 1).astype(int)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the input features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Reshape input data to match the expected input shape of the 1D CNN
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

# Data augmentation based on output values
augmented_X_train = []
augmented_y_train = []

for i in range(len(X_train)):
    augmented_X_train.append(X_train[i])
    augmented_y_train.append(y_train.iloc[i])

    # Augment data for positive class (1)
    if y_train.iloc[i] == 1:
        for _ in range(5):  # Augment 5 times for positive class
            augmented_X_train.append(X_train[i])
            augmented_y_train.append(y_train.iloc[i])

# Convert augmented data back to NumPy arrays
augmented_X_train = np.array(augmented_X_train)
augmented_y_train = np.array(augmented_y_train)

# Define the 1D CNN model
model = Sequential()
model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Implement early stopping to prevent overfitting
early_stopping = EarlyStopping(patience=5, restore_best_weights=True)

# Train the model with augmented data
model.fit(augmented_X_train, augmented_y_train, epochs=20, batch_size=32,
          validation_data=(X_test, y_test), callbacks=[early_stopping])

# Evaluate the model on the testing data
accuracy = model.evaluate(X_test, y_test, verbose=0)[1]
print(f'Accuracy: {accuracy * 100:.2f}%')


Epoch 1/20
260/260 [==============================] - 3s 5ms/step - loss: 0.0562 - accuracy: 0.9869 - val_loss: 0.0697 - val_accuracy: 0.9836
Epoch 2/20
260/260 [==============================] - 1s 4ms/step - loss: 0.0146 - accuracy: 0.9961 - val_loss: 0.0682 - val_accuracy: 0.9883
Epoch 3/20
260/260 [==============================] - 1s 5ms/step - loss: 0.0127 - accuracy: 0.9963 - val_loss: 0.0671 - val_accuracy: 0.9883
Epoch 4/20
260/260 [==============================] - 1s 4ms/step - loss: 0.0104 - accuracy: 0.9969 - val_loss: 0.0774 - val_accuracy: 0.9859
Epoch 5/20
260/260 [==============================] - 1s 4ms/step - loss: 0.0100 - accuracy: 0.9964 - val_loss: 0.0717 - val_accuracy: 0.9883
Epoch 6/20
260/260 [==============================] - 1s 4ms/step - loss: 0.0077 - accuracy: 0.9976 - val_loss: 0.0865 - val_accuracy: 0.9859
Epoch 7/20
260/260 [==============================] - 1s 4ms/step - loss: 0.0066 - accuracy: 0.9982 - val_loss: 0.0852 - val_accuracy: 0.9836
Epoch 

In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, LSTM
from tensorflow.keras.callbacks import EarlyStopping

# Load your dataset into a DataFrame (assuming it's in a CSV file)
df = pd.read_csv('data/CTG.csv')

# Drop rows with null values
df.dropna(inplace=True)

# Define input features and target variable
X = df.drop(columns=['FileName', 'Date', 'SegFile', 'NSP'])
y = df['NSP']

# Convert labels to binary (0 and 1)
y = (y == 1).astype(int)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the input features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Reshape input data to match the expected input shape of the models
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

# Define model names and corresponding model architectures
models = {
    "Feedforward Neural Network (FNN)": Sequential([
        Flatten(input_shape=(X_train.shape[1], 1)),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(64, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ]),
    "Long Short-Term Memory (LSTM) Network": Sequential([
        LSTM(64, input_shape=(X_train.shape[1], 1), return_sequences=True),
        LSTM(64),
        Dense(1, activation='sigmoid')
    ]),
    "Convolutional Neural Network (CNN)": Sequential([
        Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)),
        MaxPooling1D(pool_size=2),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])
}

# Iterate through each model, train, and evaluate
for model_name, model in models.items():
    # Compile the model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    # Implement early stopping to prevent overfitting
    early_stopping = EarlyStopping(patience=5, restore_best_weights=True)

    # Train the model
    model.fit(X_train, y_train, epochs=2, batch_size=32,
              validation_data=(X_test, y_test), callbacks=[early_stopping])

    # Evaluate the model on the testing data
    accuracy = model.evaluate(X_test, y_test, verbose=0)[1]
    print(f'{model_name} - Accuracy: {accuracy * 100:.2f}%')


Epoch 1/2
54/54 [==============================] - 1s 6ms/step - loss: 0.4004 - accuracy: 0.8276 - val_loss: 0.1445 - val_accuracy: 0.9765
Epoch 2/2
54/54 [==============================] - 0s 3ms/step - loss: 0.1327 - accuracy: 0.9682 - val_loss: 0.0799 - val_accuracy: 0.9836
Feedforward Neural Network (FNN) - Accuracy: 98.36%
Epoch 1/2
54/54 [==============================] - 8s 47ms/step - loss: 0.4420 - accuracy: 0.8171 - val_loss: 0.2506 - val_accuracy: 0.9507
Epoch 2/2
54/54 [==============================] - 2s 32ms/step - loss: 0.1264 - accuracy: 0.9618 - val_loss: 0.0858 - val_accuracy: 0.9765
Long Short-Term Memory (LSTM) Network - Accuracy: 97.65%
Epoch 1/2
54/54 [==============================] - 1s 8ms/step - loss: 0.2195 - accuracy: 0.9265 - val_loss: 0.0829 - val_accuracy: 0.9812
Epoch 2/2
54/54 [==============================] - 0s 4ms/step - loss: 0.0583 - accuracy: 0.9847 - val_loss: 0.0740 - val_accuracy: 0.9812
Convolutional Neural Network (CNN) - Accuracy: 98.12%
